In [4]:
import numpy as np

import pandas as pd

import tensorflow as tf 

import matplotlib.pyplot as plt 

In [2]:
df = pd.read_csv('MiningProcess_Flotation_Plant_Database_hour.csv')
df.set_index('date', inplace=True)

In [3]:
df.head()

,% Iron Feed,% Silica Feed,Starch Flow,Amina Flow,Ore Pulp Flow,Ore Pulp pH,Ore Pulp Density,p-Flotation Column 01 Air Flow,p-Flotation Column 02 Air Flow,p-Flotation Column 03 Air Flow,...,p-Flotation Column 07 Air Flow,p-Flotation Column 01 Level,p-Flotation Column 02 Level,p-Flotation Column 03 Level,p-Flotation Column 04 Level,p-Flotation Column 05 Level,p-Flotation Column 06 Level,p-Flotation Column 07 Level,% Iron Concentrate,% Silica Concentrate
date,,,,,,,,,,,,,,,,,,,,,
2017-03-10 01:00:00,55.2,16.98,3162.625026,578.786678,398.753368,10.113487,1.729558,251.166672,250.226086,250.178287,...,250.208184,450.383776,446.891845,450.474523,449.912259,455.792161,464.383310,450.532747,66.91,1.31
2017-03-10 02:00:00,55.2,16.98,3133.256389,537.219661,399.871822,10.129742,1.667784,249.880589,250.214050,250.033317,...,249.897572,449.373361,450.249356,450.081222,450.328806,448.722983,455.501528,451.387700,67.06,1.11
2017-03-10 03:00:00,55.2,16.98,3479.482944,591.906744,398.763806,10.048403,1.732711,250.161328,250.104167,250.046350,...,250.484183,449.972878,450.868711,450.901822,451.145822,451.134189,459.981311,450.296722,66.97,1.27
2017-03-10 04:00:00,55.2,16.98,3228.036436,593.170106,399.866983,9.918614,1.731056,250.208772,250.204761,250.120861,...,250.157622,487.940706,491.462111,487.387206,494.528183,495.664011,502.763850,494.939889,66.75,1.36
2017-03-10 05:00:00,55.2,16.98,3327.280739,619.710806,399.615089,9.746029,1.765879,249.917800,250.160494,250.013500,...,250.078639,549.031539,549.983156,549.459572,549.975483,549.512533,560.696300,550.271772,66.63,1.34


In [5]:
# Parameters 
learning_rate = 0.001 
training_epochs = 20 
batch_size = 100 
display_step = 1 

In [6]:
# Network Parameters 
n_hidden_1 = 256 

In [8]:
df = df.drop(['% Iron Concentrate'], axis=1)

In [9]:
Y = df['% Silica Concentrate']
X = df.drop(['% Silica Concentrate'], axis=1)

In [10]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_scaled = pd.DataFrame(min_max_scaler.fit_transform(X), columns=X.columns)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)

In [13]:
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(3277, 21) (820, 21) (3277,) (820,)


MLP Model


In [56]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense 
from math import sqrt  


n_features = X_train.shape[1]

# define model
model = Sequential()
model.add(Dense(21, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
model.add(Dense(8, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1))

# compile the model
model.compile(optimizer='adam', loss='mse')
# fit the model
model.fit(X_train, Y_train, epochs=30, batch_size=32, verbose=0)
# evaluate the model
error = model.evaluate(X_test, Y_test, verbose=0)
print('MSE: %.3f, RMSE: %.3f' % (error, sqrt(error)))
# make a prediction
row = [55.2,16.98,3162.625026,578.786678,398.753368,10.113487,1.729558,251.166672,250.226086,250.178287,295.0960,306.400,251.232529,250.208184,450.383776,446.891845,450.474523,449.912259,455.792161,464.38331,450.532747]
yhat = model.predict([row])
print('Predicted: %.3f' % yhat)



MSE: 1.064, RMSE: 1.032
Predicted: 1605.545


In [57]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 21)                462       
_________________________________________________________________
dense_30 (Dense)             (None, 8)                 176       
_________________________________________________________________
dense_31 (Dense)             (None, 1)                 9         
Total params: 647
Trainable params: 647
Non-trainable params: 0
_________________________________________________________________


In [49]:
from sklearn.neural_network import MLPRegressor
regr = MLPRegressor(random_state=1, max_iter=500).fit(X_train, Y_train)

regr.predict(X_test)
#regr.score(X_test, Y_test)
#row2=np.asarray(row)
#row1=row2.reshape(1, -1)
#regr.predict(row1)

array([-547.42684558])

In [50]:
regr.score(X_test, Y_test)

0.2726120297352821

In [51]:
self.result=self.regr.predict(row1)

NameError: name 'self' is not defined

In [52]:
result = regr.predict(row1)

In [53]:
result


array([-547.42684558])